In [2]:
from langchain_openai import ChatOpenAI

In [4]:
llm = ChatOpenAI()

In [6]:
from typing import TypedDict, Annotated
from langgraph.graph import Graph, StateGraph
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 상태 정의
class State(TypedDict):
    input: str
    emotion: str
    response: str

# LLM 모델 초기화 (OpenAI API 키가 필요합니다)
llm = ChatOpenAI()

# 감정 분석 함수
def analyze_emotion(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        "당신은 감정 분석가입니다. 다음 텍스트의 감정을 '긍정', '중립', '부정' 중 하나로 분석하세요: {input}"
    )
    chain = prompt | llm
    result = chain.invoke({"input": state["input"]})
    state["emotion"] = result.content
    return state

# 응답 생성 함수
def generate_response(state: State) -> State:
    prompt = ChatPromptTemplate.from_template(
        """당신은 친절한 AI 어시스턴트입니다. 사용자의 입력과 감정에 기반하여 적절한 응답을 생성하세요.
        입력: {input}
        감정: {emotion}
        응답:"""
    )
    chain = prompt | llm
    result = chain.invoke({"input": state["input"], "emotion": state["emotion"]})
    state["response"] = result.content
    return state

# 그래프 정의
workflow = StateGraph(State)

# 노드 추가
workflow.add_node("emotion_analyzer", analyze_emotion)
workflow.add_node("response_generator", generate_response)

# 엣지 추가
workflow.set_entry_point("emotion_analyzer")
workflow.add_edge("emotion_analyzer", "response_generator")
workflow.set_finish_point("response_generator")

# 그래프 컴파일
chain = workflow.compile()

# 실행 함수
def run_conversation(user_input: str):
    state = {"input": user_input}
    final_state = chain.invoke(state)
    return final_state["response"]

# 사용 예
if __name__ == "__main__":
    user_input = "오늘 날씨가 정말 좋아요!"
    response = run_conversation(user_input)
    print(f"사용자: {user_input}")
    print(f"AI: {response}")

사용자: 오늘 날씨가 정말 좋아요!
AI: 네, 날씨가 좋은 날은 기분이 좋아지죠! 즐거운 하루 보내세요. 😊
